# Download Data

In [1]:
import json
import billboard
from datetime import datetime
import time
from typing import List

def to_date(date: str) -> datetime:
    return datetime.strptime(date, '%Y-%m-%d')

def retrieve_previous_charts(chart, end_date: datetime) -> List[billboard.ChartData]:
    while to_date(chart.previousDate) > end_date:
        chart = billboard.ChartData(chart.name, chart.previousDate, fetch=True)
        yield chart
        

In [10]:
CHART_NAME = 'hot-100'
start = '1997-10-05'
end = datetime(1995, 2, 26)
chart1 = billboard.ChartData(CHART_NAME, date=start)

BillboardParseException: Failed to parse top track title

In [11]:
charts = []
for chart in retrieve_previous_charts(chart1, end):
    print(chart.date, end=" ")
    charts.append(chart)
    time.sleep(1)

BillboardParseException: Failed to parse top track title

# Process Data

In [4]:
def get_songs(charts: List[billboard.ChartData]) -> List[billboard.ChartEntry]:
    """ Remove duplicate songs from a chart list based on str(song) """
    songs = {str(song): song for chart in charts for song in chart}
    return list(songs.values())
    
def is_collaboration(song: billboard.ChartEntry):
    return any(x in song.artist for x in ["Featuring", '&'])

def song_data(song: billboard.ChartEntry) -> dict:
    return {**song.__dict__, 'identifier': str(song)}


# Get song data, remove duplicates from the charts
songs = get_songs(charts)
songs_data = list(map(song_data, songs))

# Insert into Database

In [5]:
# DB CONFIG
from pymongo import MongoClient

client = MongoClient('mongo', 27017)
db = client.music_db
songs_collection = db.top_songs

In [6]:
# Bulk Insert
songs_collection.insert_many(songs_data)

In [7]:
results = list(songs_collection.find())
len(results)

8124

In [ ]:
results[:10]